## Import libaries

In [1]:
import os
import sys
assert os.path.exists('../src'), f"[ERROR] The path src not detected in the parent directory '{os.getcwd()}'."

if os.getcwd().endswith('/notebooks'):
    os.chdir('..')
    sys.path.append('./src')

print(f'[INFO] Current Directory: "{os.getcwd()}".')

[INFO] Current Directory: "/home/iceking/Desktop/22SS-TUM Lecture Docs/3. Praktikum (IN2106, IN4249)/Repo/topic-modeling-advancements".


In [9]:
%load_ext autoreload
%autoreload 2


from src import preprocessor
from src.utils import load_documents
import nltk
import json
import ast
import pandas as pd
import time
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from src.bulk_runner import OUTPUT_FOLDER
from src.bulk_runner import main_runner

In [12]:
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)

In [30]:
combinations = []

for comb_filename in ['combinations.txt','combinations_ctm.txt']:
    with open(comb_filename) as comb_file:
        for i,item in enumerate(comb_file.readlines()):
            #item = item.replace("'",'"')
            x= ast.literal_eval(item)
            combinations.append(x)

In [ ]:
    dfs_merged=pd.DataFrame()
    for i,configs in enumerate(combinations):
        print(f'[INFO] Configs #{i}/{len(combinations)}: {configs}')
        try:
            df_output_topic_word = main_runner(configs=configs)
            dfs_merged = pd.concat([dfs_merged,df_output_topic_word])
            pd.concat(dfs_merged).to_csv(f'./{OUTPUT_FOLDER}/merged.csv')
        except Exception:
            print('[WARN] Current execution gave an error!')
            time.sleep(10)
            continue


2022-07-24 02:06:22,329 - top2vec - INFO - Pre-processing documents for training
2022-07-24 02:06:22,329 INFO:Pre-processing documents for training


[INFO] Configs #0/1974: {'algorithm_args': {'embedding_model': 'doc2vec', 'min_count': 40, 'algorithm': 'top2vec', 'num_topics': 4, 'doc2vec_speed': 'learn', 'umap_args': {'n_neighbors': 15, 'n_components': 5, 'metric': 'cosine', 'random_state': 42}, 'hdbscan_args': {'min_cluster_size': 10, 'metric': 'euclidean', 'cluster_selection_method': 'eom'}}, 'preprocessing_funcs': [], 'dataset': 'crisis_12'}
[WARN] [PREPROCESSOR] Preprocessing functions are empty or None, given:"[]", preprocessing is skipped.
[INFO] Running with 4 topics.
[INFO] Top2Vec Parameters:
{
    "embedding_model": "doc2vec",
    "min_count": 40,
    "algorithm": "top2vec",
    "num_topics": 4,
    "doc2vec_speed": "learn",
    "umap_args": {
        "n_neighbors": 15,
        "n_components": 5,
        "metric": "cosine",
        "random_state": 42
    },
    "hdbscan_args": {
        "min_cluster_size": 10,
        "metric": "euclidean",
        "cluster_selection_method": "eom"
    },
    "data_name": "crisis_12",
  

2022-07-24 02:06:22,563 - top2vec - INFO - Creating joint document/word embedding
2022-07-24 02:06:22,563 INFO:Creating joint document/word embedding
2022-07-24 02:06:43,957 - top2vec - INFO - Creating lower dimension embedding of documents
2022-07-24 02:06:43,957 INFO:Creating lower dimension embedding of documents
2022-07-24 02:07:26,149 - top2vec - INFO - Finding dense areas of documents
2022-07-24 02:07:26,149 INFO:Finding dense areas of documents
2022-07-24 02:07:26,313 - top2vec - INFO - Finding topics
2022-07-24 02:07:26,313 INFO:Finding topics


[INFO] Original (Non-reduced) Number of Topics: 4.
[WARN] # of topics is pre-specified but non_reduced_num_topics <= num_topics, so not reduced!
   > non_reduced_num_topics:4, given num_topics:4!
[INFO] Topic #00:
     > From Reduced Model:False.
     > Topic Size:5334.
     > Topic Words: ['through' 'love' 'out' 'watch' 'some' 'day' 'too' 'first' 'been' 'no'
		 'good' 'affected' 'gonna' 'show' 'city' 'water' 'damn' 'right' 'since'
		 'really' 'areas' 'know' 'weather' 'lmao' 'get' 'man' 'thoughts' 'can'
		 'breaking' 'why' 'bad' 'stop' 'like' 'disaster' 'be' 'hit' 'other'
		 'around' 'effect' 'little' 'amp' 'cyclonedebbie' 'also' 'again' 'us'
		 'another' 'fucking' 'he' 'safe' 'news']
     > Topic Word Scores: [0.9957713  0.9953993  0.99530977 0.99517995 0.9950213  0.99501574
		 0.9948502  0.99465805 0.994607   0.99450254 0.99442506 0.9942891
		 0.994256   0.9940499  0.99395394 0.99380696 0.99365807 0.993561
		 0.99355847 0.9935176  0.99346924 0.99325305 0.993179   0.99310005
		 0.9929

[0724/020730.572290:WARNING:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0724/020730.583968:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[0724/020731.039609:INFO:headless_shell.cc(660)] Written to file /home/iceking/Desktop/22SS-TUM Lecture Docs/3. Praktikum (IN2106, IN4249)/Repo/topic-modeling-advancements/output/1658621182_top2vec/vis_topic_similarity_matrix.png.


[INFO] Created Topic Similarity Matrix successfully.
[INFO] Created Top Words Barchart Visualization successfully.
[INFO] Creating UMAP 2D Scatter Plot..
[INFO] UMAP Arguments for Visualization:{'n_neighbors': 15, 'n_components': 2, 'metric': 'cosine', 'random_state': 42}
[INFO] UMAP Model is being fitted..
